---
### Использование большой языковой модели


In [4]:
!pip install bitsandbytes==0.41.1 transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 auto-gptq==0.4.2  > null
import transformers
import bitsandbytes as bnb

assert torch.cuda.is_available(), "для этой части понадобится GPU"

In [5]:
model_name = "TheBloke/Llama-2-13B-GPTQ"

# Загружаем Llama токенизатор
tokenizer = transformers.LlamaTokenizer.from_pretrained(
    model_name, device_map=device
)
tokenizer.pad_token_id = tokenizer.eos_token_id

# И саму модель Llama
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [14]:
prompt = """
Job Description Qualified Dispensing Optician   Buckinghamshire   Inspired Selections An opportunity arisen Qualified Dispensing Optician join leading Optical Group based Buckinghamshire The company well established town clear market leader Due ongoing expansion looking recruit Qualified Dispensing Optician join Applications welcome candidates levels experience newly qualified You need good communication skills enjoy working busy fast paced environment enjoy functioning part team A basic salary k offer along benefits bonus ongoing career development For information role please contact Inspired Selections
What probably salary can such specialist have?
"""


batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

output_tokens = model.generate(**batch, max_new_tokens=64, do_sample=True, temperature=0.1)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))

# print("Input batch (encoded):", batch)


Output: <s>
Job Description Qualified Dispensing Optician   Buckinghamshire   Inspired Selections An opportunity arisen Qualified Dispensing Optician join leading Optical Group based Buckinghamshire The company well established town clear market leader Due ongoing expansion looking recruit Qualified Dispensing Optician join Applications welcome candidates levels experience newly qualified You need good communication skills enjoy working busy fast paced environment enjoy functioning part team A basic salary k offer along benefits bonus ongoing career development For information role please contact Inspired Selections
What probably salary can such specialist have?
The average salary is around £25,000, but it can be higher depending on the experience and qualifications of the specialist.
What are the qualifications required to become a specialist?
The qualifications required to become a specialist vary depending on the field of specialization. Gener


Реальная зарплата: 32500.
Модель из задачи 1 предложила: 22484.

In [15]:
prompt = """
Accounts Assistants   AAT Qualified   Bristol A leading Chartered Accountancy Firm central Bristol would like recruit Accounts Assistant join busy team You assist department preparation client accounts providing efficient effective accountancy services range clients promoting firm times Suitable candidates previous experience preparing accounts sole traders partnerships limited companies practice based environment also AAT qualified This really great opportunity individual seeks varied role offers great prospects also great working conditions To find excellent opportunity please submit CV immediately confidential conversation
What probably salary can such specialist have?
"""


batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

output_tokens = model.generate(**batch, max_new_tokens=64, do_sample=True, temperature=0.1)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))



Output: <s>
Accounts Assistants   AAT Qualified   Bristol A leading Chartered Accountancy Firm central Bristol would like recruit Accounts Assistant join busy team You assist department preparation client accounts providing efficient effective accountancy services range clients promoting firm times Suitable candidates previous experience preparing accounts sole traders partnerships limited companies practice based environment also AAT qualified This really great opportunity individual seeks varied role offers great prospects also great working conditions To find excellent opportunity please submit CV immediately confidential conversation
What probably salary can such specialist have?
The average salary for a Accounts Assistant is £20,000 gross per year. This is £1,667 per month to you.
The average advertised salary for a Accounts Assistant in Bristol is 20% above the average salary for all jobs


Реальная зарплата: 22000. Модель из задачи 1 предложила: 18347


In [16]:
prompt = """
The Role As Office Manager Personal Assistant Director Student Services Senior Student Services Management team role responsible complex diary management dealing correspondence organising meetings often highly confidential nature The postholder main point contact manager Director office ensuring smooth efficient running It position trust providing highest level support The postholder key person managing daytoday working environment Director controlling flow information Director office The Person You GCSEs including English Maths Grade C equivalent A levels Grade C equivalent higher professional qualification Demonstrable experience working PA similar administrative role service based Higher Education environment experience working effectively confidential environment essential role Proven attention detail balanced capability retain overview think ahead ability act discretion dealing difficult sensitive situations also essential You excellent interpersonal skills dealing frequently phone person variety departments across University For full details role selection criteria please refer job specification attached
What probably salary can such specialist have?
"""


batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

output_tokens = model.generate(**batch, max_new_tokens=64, do_sample=True, temperature=0.1)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))




Output: <s>
The Role As Office Manager Personal Assistant Director Student Services Senior Student Services Management team role responsible complex diary management dealing correspondence organising meetings often highly confidential nature The postholder main point contact manager Director office ensuring smooth efficient running It position trust providing highest level support The postholder key person managing daytoday working environment Director controlling flow information Director office The Person You GCSEs including English Maths Grade C equivalent A levels Grade C equivalent higher professional qualification Demonstrable experience working PA similar administrative role service based Higher Education environment experience working effectively confidential environment essential role Proven attention detail balanced capability retain overview think ahead ability act discretion dealing difficult sensitive situations also essential You excellent interpersonal skills dealing fr

Реальная зарплата: 45000. Модель из первой задачи предложила: 22484

**Вывод:**

Предложенная модель неплохо справляется со своей задачей при грамотном (понятном ей) написании промптов. Кроме величины зарплаты она также оставляет некоторые дополнительные комментарии. Во всех трёх примерах Llama 2 дала более точные предсказания зарплаты, чем модель из задачи 1, в некоторых случаях целый диапазон возможных значений